# Importing packages and files

In [1]:
from importlib import reload
import Scenario
import Technology
import statistics
from pathlib import Path
reload(Scenario)
reload(Technology)
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import copy
import sys
import plotly.graph_objects as go
from Scenario import Scenario
from Technology import Technology
import requests
from io import StringIO
from scipy.special import logsumexp
from pyomo.environ import *
from pyomo.environ import RangeSet
from pyomo.environ import value

In [2]:
data_path = 'data'

In [ ]:
savefigs = True

# Constructing numerical matrices and vectors for LCA and TEA

In [ ]:
# Helper to read a CSV from a public GitHub raw URL
def read_csv_public(url: str) -> pd.DataFrame:
    """Load a CSV hosted on a public GitHub raw link."""
    return pd.read_csv(url, header=None)

# Data locations
A_url = "https://raw.githubusercontent.com/FarshidNazemi/barrier-film-software/refs/heads/main/data/REMADE-tool-A.csv"
B_url = "https://raw.githubusercontent.com/FarshidNazemi/barrier-film-software/refs/heads/main/data/REMADE-tool-B.csv"
C_url = "https://raw.githubusercontent.com/FarshidNazemi/barrier-film-software/refs/heads/main/data/REMADE-tool-C.csv"

# Read data
A_df_org    = read_csv_public(A_url)
A_df_design = A_df_org.copy()
A_df        = A_df_org.copy()

B_df_org    = read_csv_public(B_url)
B_df_design = B_df_org.copy()
B_df        = B_df_org.copy()

C_df_org    = read_csv_public(C_url)
C_df_design = C_df_org.copy()
C_df        = C_df_org.copy()